In [ ]:
import pandas as pd
import re

In [ ]:
gt_path_list = []
gt_list = []
pred_path_list = []
pred_list = []
pred_score_list = []

In [ ]:
gt_file = open('./log_results/gt.txt', 'r')
for ground_truth in gt_file.readlines():
    path, gt = ground_truth.rstrip().split()
    gt_path_list.append(path)
    gt_list.append(gt)

In [ ]:
prediction_file = open('./log_results/log_demo_result_case_sensitive.txt', 'r')
for prediction in prediction_file.readlines():
    path, pred, score = prediction.rstrip().split()
    pred_path_list.append(path)
    pred_list.append(pred)
    pred_score_list.append(float(score))

In [ ]:
len(gt_path_list) == len(pred_path_list)

In [ ]:
gt_df = pd.DataFrame({'path': gt_path_list, 'gt': gt_list})
pred_df = pd.DataFrame({'path': pred_path_list, 'pred': pred_list, 'confidence_score': pred_score_list})

In [ ]:
concat_df = pd.merge(gt_df, pred_df, how='inner', on='path')

In [ ]:
# 한글 포함한 데이터 제외하기 위한 regex
reg = re.compile(r'[a-zA-Z0-9]')

In [ ]:
def count_stats(df):
    hangul_cnt = 0
    correct_cnt = 0
    wrong_cnt = 0
    for idx, row in df.iterrows():
        if not reg.match(row['gt']):
            hangul_cnt += 1
            continue
        if row['gt'] == row['pred']:
            correct_cnt += 1
        else:
            wrong_cnt += 1
    tot_cnt = hangul_cnt + correct_cnt + wrong_cnt
    score_w_hangul = correct_cnt/tot_cnt
    score = correct_cnt/(correct_cnt+wrong_cnt)
    return tot_cnt, hangul_cnt, correct_cnt, wrong_cnt, score_w_hangul, score

### Filter with confidence scores

In [ ]:
filter_05 = concat_df[concat_df['confidence_score'] >= 0.5]
filter_06 = concat_df[concat_df['confidence_score'] >= 0.6]
filter_07 = concat_df[concat_df['confidence_score'] >= 0.7]
filter_075 = concat_df[concat_df['confidence_score'] >= 0.75]
filter_08 = concat_df[concat_df['confidence_score'] >= 0.8]

In [ ]:
print(count_stats(concat_df))
print(count_stats(filter_05))
print(count_stats(filter_06))
print(count_stats(filter_07))
print(count_stats(filter_075))
print(count_stats(filter_08))

In [ ]:
filter_08[filter_08['gt'] != filter_08['pred']]

### Jaccord

In [ ]:
def jaccord_similarity(x, y):
    numerator = len(set.intersection(*[set(x), set(y)]))
    denominator = len(set.union(*[set(x), set(y)]))
    return round((numerator/float(denominator)), 3)

def custom_jaccord_similarity(x, y):
    numerator = len(set.intersection(*[set(x), set(y)]))
    denominator = len(set(y))
    return round((numerator/float(denominator)), 3)

test = "test"
pred = "pred"
print(jaccord_similarity(test, pred))
print(custom_jaccord_similarity(test, pred))

In [ ]:
def cal_jaccord(df):
    hangul_cnt = 0
    jaccord_correct = 0
    jaccord_wrong = 0
    custom_jaccord_correct = 0
    custom_jaccord_wrong = 0
    for row in df.iterrows():
        jaccord = jaccord_similarity(row[1]['gt'], row[1]['pred'])
        custom_jaccord = custom_jaccord_similarity(row[1]['gt'], row[1]['pred'])
        if not reg.match(row[1]['gt']):
            hangul_cnt += 1
            continue
        if jaccord >= 0.5:
            jaccord_correct += 1
        else:
            jaccord_wrong += 1
        if custom_jaccord >= 0.5:
            custom_jaccord_correct += 1
        else:
            custom_jaccord_wrong += 1
    return hangul_cnt, jaccord_correct, jaccord_wrong, jaccord_correct + jaccord_wrong, custom_jaccord_correct, custom_jaccord_wrong, custom_jaccord_correct + custom_jaccord_wrong


In [ ]:
print(cal_jaccord(concat_df))
print(cal_jaccord(filter_05))
print(cal_jaccord(filter_06))
print(cal_jaccord(filter_07))
print(cal_jaccord(filter_075))
print(cal_jaccord(filter_08))

# Not Case-Sensitive

In [ ]:
gt_lower_list = []
pred_lower_list = []
pred_score_lower_list = []

In [ ]:
gt_file = open('./log_results/gt.txt', 'r')
for ground_truth in gt_file.readlines():
    _, gt = ground_truth.rstrip().split()
    gt_lower_list.append(gt.lower())

In [ ]:
prediction_lower_file = open('./log_results/log_demo_result_no_case.txt', 'r')
for prediction in prediction_lower_file.readlines():
    _, pred, score = prediction.rstrip().split()
    pred_lower_list.append(pred)
    pred_score_lower_list.append(float(score))

In [ ]:
len(gt_lower_list)

In [ ]:
gt_lower_df = pd.DataFrame({'path': gt_path_list, 'gt': gt_lower_list})
pred_lower_df = pd.DataFrame({'path': pred_path_list, 'pred': pred_lower_list, 'confidence_score': pred_score_lower_list})

In [ ]:
lower_concat_df = pd.merge(gt_lower_df, pred_lower_df, how='inner', on='path')

In [ ]:
lower_filter_05 = lower_concat_df[lower_concat_df['confidence_score'] >= 0.5]
lower_filter_06 = lower_concat_df[lower_concat_df['confidence_score'] >= 0.6]
lower_filter_07 = lower_concat_df[lower_concat_df['confidence_score'] >= 0.7]
lower_filter_075 = lower_concat_df[lower_concat_df['confidence_score'] >= 0.75]
lower_filter_08 = lower_concat_df[lower_concat_df['confidence_score'] >= 0.8]

In [ ]:
print(count_stats(lower_concat_df))
print(count_stats(lower_filter_05))
print(count_stats(lower_filter_06))
print(count_stats(lower_filter_07))
print(count_stats(lower_filter_075))
print(count_stats(lower_filter_08))

In [ ]:
lower_filter_08[lower_filter_08['gt'] != lower_filter_08['pred']]